In [7]:
import torch
from diffusers import DiffusionPipeline

# Load the smallest LDM model
pipe = DiffusionPipeline.from_pretrained("CompVis/ldm-text2im-large-256", torch_dtype=torch.float32)

# Move model to CPU
pipe.to("cpu")

print("Model running on:", pipe.device)



Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch C:\Users\97798\.cache\huggingface\hub\models--CompVis--ldm-text2im-large-256\snapshots\30de525ca11a880baea4962827fb6cb0bb268955\unet: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\97798\.cache\huggingface\hub\models--CompVis--ldm-text2im-large-256\snapshots\30de525ca11a880baea4962827fb6cb0bb268955\unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch C:\Users\97798\.cache\huggingface\hub\models--CompVis--ldm-text2im-large-256\snapshots\30de525ca11a880baea4962827fb6cb0bb268955\vqvae: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\97798\.cache\huggingface\hub\models--CompVis--ldm-text2im-large-256\snapshots\30de525ca11a880baea4962827fb6cb0bb268955\vqvae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The config attributes {'timestep_values': None} were p

Model running on: cpu


In [8]:
prompt = "A cute cat sticker with bold outlines, cartoon style"
image = pipe(prompt, guidance_scale=7.5).images[0]

# Save & Show Image
image.save("cpu_generated_sticker.png")
image.show()


  0%|          | 0/50 [00:00<?, ?it/s]

In [7]:
scheduler = DDPMScheduler(num_train_timesteps=1000, beta_start=0.0001, beta_end=0.02)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

In [8]:
def train_diffusion(epochs=10):
    model.train()
    for epoch in range(epochs):
        for step, (images, _) in enumerate(dataloader):
            images = images.to(device)
            timesteps = torch.randint(0, scheduler.num_train_timesteps, (images.shape[0],), device=device).long()
            noise = torch.randn_like(images).to(device)
            noisy_images = scheduler.add_noise(images, noise, timesteps)

            predicted_noise = model(noisy_images, timesteps).sample
            loss = nn.functional.mse_loss(predicted_noise, noise)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if step % 100 == 0:
                print(f"Epoch {epoch+1}, Step {step}, Loss: {loss.item():.4f}")

    print("Training complete.")



In [9]:
def generate_sticker(num_images=1):
    model.eval()
    with torch.no_grad():
        noise = torch.randn((num_images, 3, 64, 64), device=device)
        scheduler.set_timesteps(1000)

        for t in scheduler.timesteps:
            noise = model(noise, t).sample
            noise = scheduler.step(noise, t, noise).prev_sample

        images = (noise.clamp(-1, 1) + 1) / 2  
        images = images.cpu().permute(0, 2, 3, 1).numpy()

        for i, img in enumerate(images):
            img = (img * 255).astype('uint8')
            Image.fromarray(img).save(f"sticker_{i}.png")

    print("Sticker generation complete.")


In [10]:
train_diffusion(epochs=5)  
generate_sticker(num_images=5)


c:\Users\97798\AppData\Local\Programs\Python\Python311\Lib\site-packages\diffusers\configuration_utils.py:140: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


Epoch 1, Step 0, Loss: 1.0970
Epoch 1, Step 100, Loss: 0.1109
Epoch 1, Step 200, Loss: 0.1018
Epoch 1, Step 300, Loss: 0.0433
Epoch 1, Step 400, Loss: 0.0461
Epoch 1, Step 500, Loss: 0.0707


KeyboardInterrupt: 